In [1]:
# IR HW5 Simple Mixture Model MAP@5000
path = './ntust-ir-2020_hw5_new/'
import numpy as np
import time
import random
from tqdm import tqdm
from numba import jit, objmode
from numba.core import types
from numba.typed import List
from collections import Counter
from math import log

In [2]:
# data of doc and query
query = List()
doc = List()
# length of each doc
doc_len = List()
# name of doc and query
query_list = []
doc_list = []
# vocabulary
vocabulary = set()
# vocabulary counting(TF)
TF = []
# BackGround Counting
bg_tf = Counter()
# BM25 query vocabulary
bm25_vocabulary = set()
# get doc files and query files
with open(path + 'query_list.txt', 'r') as fq:
    line = fq.read().splitlines()
    for l in line:
        query_list.append(l)
        f_temp = open(path + 'queries/{}.txt'.format(l))
        temp_query = f_temp.read().split()
        for v in temp_query:
            bm25_vocabulary.add(v)
            vocabulary.add(v)
        query.append(temp_query)
        f_temp.close()
with open(path + 'doc_list.txt', 'r') as fd:
    line = fd.read().splitlines()
    for l in tqdm(line):
        doc_list.append(l)
        f_temp = open(path + 'docs/{}.txt'.format(l))
        temp_doc = f_temp.read().split()
        doc.append(temp_doc)
        TF.append(dict(Counter(temp_doc)))
        bg_tf.update(temp_doc)
        doc_len.append(len(temp_doc))
        for w in temp_doc:
            vocabulary.add(w)
        f_temp.close()

100%|██████████| 30000/30000 [01:07<00:00, 445.40it/s]


In [3]:
# BackGroud variable
bg_len = np.sum(doc_len)
# All doc, query and vocabulary length
d_len = len(doc)
q_len = len(query)
w_len = len(vocabulary)
# vocabulary to list
vocabulary = list(vocabulary)
bm25_vocabulary =list(bm25_vocabulary)
bm25_len = len(bm25_vocabulary)

In [4]:
# Probability BackGround Model
P_bg = np.zeros(w_len, dtype=float)
for w in tqdm(range(w_len)):
    P_bg[w] = bg_tf[vocabulary[w]] / bg_len

100%|██████████| 154240/154240 [00:00<00:00, 265337.83it/s]


In [5]:
print(q_len, d_len, w_len)

150 30000 154240


In [6]:
# IDF
@jit(nopython=True)
def IDF(idf, doc_, bm25_voc, dl, bm25l):
    with objmode(time1 = 'f8'):
        time1 = time.perf_counter()
    for d in range(dl):
        for v in range(bm25l):
            if(bm25_voc[v] in doc_[d]):
                idf[v] += 1
    with objmode():
        print('time: {:.2f} mins\n'.format((time.perf_counter() - time1) / 60))
    return idf

In [7]:
# BM25 model variable
bm_sim = np.zeros([q_len, d_len], dtype=float) # 150 x 30000
K1 = 0.8
b = 0.7
N = d_len
avg_dl = np.average(doc_len)

In [8]:
# BM25 IDF
idf_doc = np.zeros(len(bm25_vocabulary), dtype=int)
idf_doc = IDF(idf_doc, doc, bm25_vocabulary, d_len, bm25_len)

time: 6.49 mins



In [9]:
# B25 Model
@jit(nopython=True)
def BM25(sim, k1, b_, N_, avg, dl_list, dl, ql, Doc, Query, idf_, bm_voc):
    with objmode(time1='f8'):
        time1 = time.perf_counter()
    for q in range(ql):
        for d in range(dl):
            sum = 0.0
            for w in Query[q]:
                sum_tmp = 1.0
                if(w in Doc[d]):
                    w_tf = Doc[d].count(w)
                    ni_loc = bm_voc.index(w)
                    ni = idf_[ni_loc]
                    sum_tmp = (k1 + 1) * w_tf / (k1 * ((1 - b) + b * dl_list[d] / avg) + w_tf)
                    sum_tmp *= log((N_ - ni + 0.5) / (ni + 0.5))
                sum += sum_tmp
            sim[q][d] = sum
    with objmode():
        print('time: {:.2f} mins\n'.format((time.perf_counter() - time1) / 60))
    return sim

In [10]:
bm_sim = BM25(bm_sim, K1, b, N, avg_dl, doc_len, d_len, q_len, doc, query, idf_doc, bm25_vocabulary)

time: 5.62 mins



In [11]:
# file output
fp = open("bm25.txt", "w")
fp.write("Query,RetrievedDocuments\n")
for q in range(q_len):
    fp.write('{},'.format(query_list[q]))
    for s in np.argsort(-bm_sim[q])[:5000]:
        fp.write(doc_list[s] + ' ')
    fp.write('\n')
fp.close()

In [12]:
# SMM variable
SMM_1 = 5000
SMM_2 = 3000
SMM_3 = 1000
Rq_1 = []
Rq_2 = []
Rq_3 = []
for q in tqdm(range(q_len)):
    for s in np.argsort(-bm_sim[q])[:SMM_1]:
        Rq_1.append(doc[s])
Rq_2 = Rq_1[:SMM_2]
Rq_3 = Rq_1[:SMM_3]

100%|██████████| 150/150 [00:02<00:00, 73.81it/s]


In [13]:
# Simple Mixture Model initialization
PSMM_temp = np.zeros(w_len, dtype=float)
P_Tsmm_temp = np.zeros(w_len, dtype=float)
# Random
r_sum = 0
for w in range(w_len):
    p_temp = random.uniform(1.0, 10.0)
    PSMM_temp[w] = p_temp
    r_sum += p_temp
for w in range(w_len):
    PSMM_temp[w] /= r_sum

In [14]:
PSMM_1 = PSMM_temp
P_Tsmm_1 = P_Tsmm_temp

In [15]:
# SMM E_step
@jit(nopython=True)
def E_step(tsmm, alpha_, psmm, p_bg, wl):
    for w in range(wl):
        tmss[w] = ((1 - alpha_) * psmm[w]) / ((1 - alpha_) * psmm[w] + alpha_ * p_bg[w])
    return tmss

In [16]:
# SMM M_step
@jit(nopython=True)
def M_step(psmm, rel_doc, Vocabulary, tmss, wl):
    den = 0
    for w in range(wl):
        for d in rel_doc:
            den += d.count(Vocabulary[w]) * tmss[w]
    for w in range(wl):
        mol = 0
        for d in rel_doc:
            mol += d.count(Vocabulary[w]) * tmss[w]
        psmm[w] = mol / den
    return psmm

In [ ]:
iter = 1
alpha = 0.8
# Processing time
# full_time = time.time()
for i in tqdm(range(iter)):
    print("E_step_{}:\n".format(i+1))
    e_start = time.time()
    P_Tsmm_1 = E_step(P_Tsmm_1, alpha, PSMM_1, P_bg, w_len)
    print("E_time: {:.2f} mins\n".format((e_start - time.time()) / 60))
    print("M_step_{}:\n".format(i+1))
    m_start = time.time()
    PSMM_1 = M_step(PSMM_1, Rq_1, vocabulary, P_Tsmm_1, w_len)
    print("M_time: {:.2f} mins\n".format((m_start - time.time()) / 60))
# print('Full Extime = {:.2f} mins\n'.format((time.time() - full_time) / 60))

In [ ]:
# KL Divergence
kl_div = np.zeros([q_len, d_len], dtype=float)